In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_22412\3609341226.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
import numpy as np
import plotly.graph_objects as go

c = 3
F = 100
n = 10
ylds = np.linspace(0.01, 0.12, 50)
prices = [
  c*np.sum((1+y/2)**np.arange(-1,-n-1,-1)) + F/(1+y/2)**n for y in ylds
]

trace = go.Scatter(
  x=ylds,
  y=prices,
  mode="lines",
  hovertemplate="yield = %{x:.1%}<br>price = $%{y.2f}<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    template="plotly_dark",
    xaxis_title="Yield",
    yaxis_title="Price",
    yaxis_tickformat=".0f",
    xaxis_tickformat=".0%",
    )
fig.update_xaxes(title_font_size=24)
fig.update_yaxes(title_font_size=24)
fig.update_layout(font_size=20)
fig.show()

In [3]:
from pandas_datareader import DataReader as pdr
import plotly.express as px

files = ["DGS" + x for x in ["1MO", "3MO", "1", "2", "3", "5", "10", "20", "30"]]
df = pdr(files, "fred", start=1920) / 100

df.index.name = "date"
df = df.reset_index()

df["month"] = df.date.dt.to_period("M").astype(str)
df = df.groupby("month").first()
df = df.drop(columns=["date"])
df = df.dropna(subset=["DGS3MO", "DGS30"])
df.columns = [1 / 12, 1 / 4, 1, 2, 3, 5, 10, 20, 30]

df = df.stack()
df = df.reset_index()
df.columns = ["month", "term", "rate"]

hover_data = dict(month=False, term=True, rate=True)
fig = px.line(
        df,
        x="term",
        y="rate",
        animation_frame="month",
        hover_name="month",
        hover_data=hover_data,
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title="Years to Maturity",
    yaxis_title="Yield",
    yaxis_tickformat=".1%",
)
fig.update_xaxes(title_font_size=24)
fig.update_yaxes(title_font_size=24)
fig.update_layout(font_size=20)
fig.update_yaxes(tickformat=".1%", range=[0, df.rate.max() + 0.001])
fig.update_xaxes(tickvals=[1, 3, 5, 10, 20, 30], range=[0, 30])
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 200
fig.show()  

In [4]:
import plotly.graph_objects as go

files = ["BAMLC0A4CBBB", "BAMLC0A3CA", "BAMLC0A2CAA", "BAMLC0A1CAAA"]
df = pdr(files, "fred", start=1920) / 100
df.index.name = "date"
df.index = df.index.to_period("M").astype(str)
df = df.groupby("date").first()
df.columns = ["BBB", "A", "AA", "AAA"]
df = df.reset_index()

trace1 = go.Scatter(
  x=df.date,
  y=df.BBB,
  mode="lines",
  hovertemplate="%{y.1%}",
  name="BBB"
)

trace2 = go.Scatter(
  x=df.date,
  y=df.A,
  mode="lines",
  hovertemplate="%{y.1%}",
  name="A"
)

trace3 = go.Scatter(
  x=df.date,
  y=df.AA,
  mode="lines",
  hovertemplate="%{y.1%}",
  name="AA"
)

trace4 = go.Scatter(
  x=df.date,
  y=df.AAA,
  mode="lines",
  hovertemplate="%{y.1%}",
  name="AAA"
)

fig = go.Figure()
for trace in [trace1, trace2, trace3, trace4]:
  fig.add_trace(trace)

fig.update_layout(
    template="plotly_dark",
    xaxis_title="Date",
    yaxis_title="Credit Spread",
    yaxis_tickformat=".0%",
    hovermode="x unified"
)
fig.update_xaxes(title_font_size=24)
fig.update_yaxes(title_font_size=24)
fig.update_layout(font_size=20)
fig.show()